## Script om 2D overstromingsbeelden van twee of meerdere modelscenario's of modellen te vergelijken
Dit script leest 2D overstromingsresultaten uit van twee (of meer) modelscenario's en berekend verschillen tussen beide

Aandachtspunten:
* voor een werkend script moet bij D-Hydro versie 2022.04 Mesh met een hoofdletter m worden geschreven. Dat dient ook in het achterliggende resultreaderSubGrid.py verwerkt te worden
* de padverwijzingen in het notebook is gericht op modellen die zijn opgeslagen en doorgerekend in de GUI (.dsproj_data\FlowFM\output\)
* gebruik van anaconda environment delft3dfmpy_2_0_3
* het script maakt gebruik van resultsreaderSubGrid.py. Dit bestand dient in dezelfde map als deze notebook geplaatst te worden.

### Inlezen van de benodigde packages

In [ ]:
#packages inlezen
from netCDF4 import Dataset, chartostring
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
import os, sys, time
import numpy as np
import geopandas as gpd
import hkvsobekpy as hkv
import resultreaderSubGrid as resultreader

from datetime import timedelta
from datetime import datetime
from datetime import datetime
from pathlib import Path
from shapely.geometry import Point
%matplotlib inline

### Functies definieren

Hier worden de functies gedefinieert waarmee de modelresultaten naar shapefiles worden geschreven

#### Wegschrijven modelresultaat naar shapefile
Functie om fou en map bestand om te zetten naar shape met daarin maximum waterdiepte, tijdstip (uren) waarop cel nat wordt (meer dan 2 cm diepte), tijdstip waarop cel weer droog wordt (minder dan 2 cm diepte), duur van nat zijn en waterdiepte aan einde simulatie.

In [ ]:
def Fou2Kaart(Werkmap,ResMap,ID):
    FourrierFile=ResMap+"/output/"+'FlowFM_fou.nc'
    MapFile=ResMap+"/output/"+'FlowFM_map.nc'
    Folder=ResMap
    #Fourrier uitlezen
    results = resultreader.Results(FourrierFile)
    DepthM=results.get_data('Mesh2d_fourier001_max_depth').data
    DepthM[DepthM<0.01]=-9999
    #Map bestand uitlezen
    results = resultreader.Results(MapFile)
    DepthT=results.get_data('Mesh2d_waterdepth').data
    DepthM[DepthT[1,:]>0]=-9999
    Time=results.get_data('time').data
    #Berekenen stijgsnelheid en Twet
    Twet=np.zeros(len(DepthM))    
    Tdry=np.zeros(len(DepthM))
    Tduur=np.zeros(len(DepthM))
    for j,Tijd in enumerate(Time):
        Diepte_tijdT=DepthT[j,:]
        for k,D in enumerate(Diepte_tijdT):
            if (D>0.02) and (Twet[k]==0):
                Twet[k]=Tijd
            if (D<0.02) and (Twet[k]>0):        
                Tdry[k]=Tijd
            else:
                Tdry[k]=Tijd
    Tduur=(Tdry-Twet)/3600   
    Dend=DepthT[-1,:]
    gdf = gpd.GeoDataFrame(
        data=np.c_[
            DepthM,
            Twet,
            Tdry,
            Tduur,
            Dend,
        ],
        columns=['wdep_max', 'Twet','Tdry','Tduur','D_einde'],
        geometry=results.get_Mesh2d_faces(as_polygon=True),
        crs={'init': 'epsg:28992'}
    )
    gdf.to_file(Werkmap+'/'+ID+'_VectorResultaten.shp')  

#### Berekenen verschil in inundatieoppervlak en waterdiepte tussen twee modellen
Functie om het vershil in waterdiepte tussen twee fou bestanden (zelfde rooster) te bepalen en om bins op te zetten die aangeven wat het verschil in geinundeerd gebied is. Bin=1 is alleen nat in fou bestand 1, Bin=2 is alleen nat in 2, Bin=3 is nat in allebei en Bin=0 is droog in allebei. Resultaat komt in shape.

In [ ]:
def DiffDiep(Werkmap,ResMap1,ID1,ResMap2,ID2):
    FourrierFile=ResMap1+"/output/"+'FlowFM_fou.nc'
    results = resultreader.Results(FourrierFile)
    DepthM_1=results.get_data('Mesh2d_fourier001_max_depth').data    
    FourrierFile=ResMap2+"/output/"+'FlowFM_fou.nc'    
    results = resultreader.Results(FourrierFile)
    DepthM_2=results.get_data('Mesh2d_fourier001_max_depth').data
    Bin=np.zeros(len(DepthM_1))
    for k,val in enumerate(Bin):
        if ((DepthM_1[k]>0.01) and (DepthM_2[k]<=0.01)):
            Bin[k]=1
        elif ((DepthM_1[k]>0.01) and (DepthM_2[k]>0.01)):
            Bin[k]=3
        elif ((DepthM_1[k]<=0.01) and (DepthM_2[k]<=0.01)):
            Bin[k]=0
        elif ((DepthM_1[k]<=0.01) and (DepthM_2[k]>0.01)):
            Bin[k]=2   
    gdf = gpd.GeoDataFrame(
        data=np.c_[
            DepthM_2-DepthM_1,
            Bin,
        ],
        columns=['DiffH','bin'],
        geometry=results.get_Mesh2d_faces(as_polygon=True),
        crs={'init': 'epsg:28992'}
    )
    name='diffDiep_'+ID1+'-'+ID2
    gdf.to_file(Werkmap+'/'+name+'.shp')              

#### Berekenen verschil in inundatieoppervlak, waterdiepte en duur van de inundatie tussen twee modellen
Functie die bepaald:
* verschil waterdiepte tussen twee fou bestanden (zelfde rooster)
* tijdsverschil tussen nat worden
* tijdsverschil tussen weer droog worden
* tijdsverschil duur inundatie
* verschil geinundeerd gebied berekenen

In [ ]:
def DiffDiepDuur(Werkmap,ResMap1,ID1,ResMap2,ID2):
    FourrierFile=ResMap1+"/output/"+'FlowFM_fou.nc'
    MapFile=ResMap1+"/output/"+'FlowFM_map.nc'
    Folder=ResMap1
    #Fourrier uitlezen
    results = resultreader.Results(FourrierFile)
    DepthM_1=results.get_data('Mesh2d_fourier001_max_depth').data
    #DepthM_1[DepthM_1<0.01]=-9999
    #Map bestand uitlezen
    results = resultreader.Results(MapFile)
    DepthT_1=results.get_data('Mesh2d_waterdepth').data
    #DepthM_1[DepthT_1[1,:]>0]=-9999
    Time=results.get_data('time').data
    #Berekenen stijgsnelheid en Twet
    Twet_1=np.zeros(len(DepthM_1))    
    Tdry_1=np.zeros(len(DepthM_1))
    Tduur_1=np.zeros(len(DepthM_1))
    for j,Tijd in enumerate(Time):
        Diepte_tijdT=DepthT_1[j,:]
        for k,D in enumerate(Diepte_tijdT):
            if (D>0.01) and (Twet_1[k]==0):
                Twet_1[k]=Tijd
            if (D<0.01) and (Twet_1[k]>0):        
                Tdry_1[k]=Tijd
            elif (D>0.01) and (Twet_1[k]>0):        
                Tdry_1[k]=Tijd            
            else:
                Tdry_1[k]=0
        #Tdry_1[Twet_1==0]=-9999
        #Twet_1[Twet_1==0]=-9999
    Tduur_1=(Tdry_1-Twet_1)/3600    
    
    FourrierFile=ResMap2+"/output/"+'FlowFM_fou.nc'
    MapFile=ResMap2+"/output/"+'FlowFM_map.nc'
    Folder=ResMap2
    #Fourrier uitlezen
    results = resultreader.Results(FourrierFile)
    DepthM_2=results.get_data('Mesh2d_fourier001_max_depth').data
    #DepthM_2[DepthM_2<0.01]=-9999
    #Map bestand uitlezen
    results = resultreader.Results(MapFile)
    DepthT_2=results.get_data('Mesh2d_waterdepth').data
    #DepthM_2[DepthT_2[1,:]>0]=-9999
    Time=results.get_data('time').data
    #Berekenen stijgsnelheid en Twet
    Twet_2=np.zeros(len(DepthM_2))    
    Tdry_2=np.zeros(len(DepthM_2))
    Tduur_2=np.zeros(len(DepthM_2))
    for j,Tijd in enumerate(Time):
        Diepte_tijdT=DepthT_2[j,:]
        for k,D in enumerate(Diepte_tijdT):
            if (D>0.01) and (Twet_2[k]==0):
                Twet_2[k]=Tijd
            if (D<0.01) and (Twet_2[k]>0):        
                Tdry_2[k]=Tijd
            elif (D>0.01) and (Twet_2[k]>0):        
                Tdry_2[k]=Tijd            
            else:
                Tdry_2[k]=0
        #Tdry_2[Twet_2==0]=-9999
        #Twet_2[Twet_2==0]=-9999
    Tduur_2=(Tdry_2-Twet_2)/3600
    
    Only1=np.zeros(len(DepthM_1))
    for k,val in enumerate(Only1):
        if ((DepthM_1[k]>0.01) and (DepthM_2[k]<=0.01)):
            Only1[k]=1
        elif ((DepthM_1[k]>0.01) and (DepthM_2[k]>0.01)):
            Only1[k]=3
        elif ((DepthM_1[k]<=0.01) and (DepthM_2[k]<=0.01)):
            Only1[k]=0
        elif ((DepthM_1[k]<=0.01) and (DepthM_2[k]>0.01)):
            Only1[k]=2
    
    gdf = gpd.GeoDataFrame(
        data=np.c_[
            DepthM_1-DepthM_2,
            Twet_1-Twet_2,
            Tdry_1-Tdry_2,
            Tduur_1-Tduur_2,
            Only1,
        ],
        columns=['wdep_diff', 'Twet_diff','Tdry_diff','Tduur_diff','bin'],
        geometry=results.get_Mesh2d_faces(as_polygon=True),
        crs={'init': 'epsg:28992'}
    )
    name='diffDiepDuur_'+ID1+'-'+ID2
    gdf.to_file(Werkmap+'/'+name+'.shp')  

### Modellen definieren
Hieronder worden:
* de locatie van de doorgerekende modellen gedefinieerd
* de locatie van het resultaat gedefineerd
* de modelscenario's opgegeven 

In [ ]:
model_pad = os.path.abspath('../03_Model') 
output_pad = os.path.abspath('../02_Validatie/2Doverstromingsbeeld')
Modellensets=['06_1D2DmodelGoorloop','07_1D2DmodelGoorloop_GHG140']

### Runnen van de functie
Hierondder worden de functies uitgevoerd en wordt het resultaat van de analyse weggeschreven

#### Modelresultaat naar shapefile per model

In [ ]:
for modelset in Modellensets:   
    Fou2Kaart(output_pad,model_pad+'/'+modelset+'.dsproj_data/FlowFM',modelset)

#### Verschil inundatieoppervlak en waterdiepte naar shapefile

In [ ]:
DiffDiep(output_pad,model_pad+'/'+Modellensets[1]+'.dsproj_data/FlowFM',Modellensets[1],model_pad+'/'+Modellensets[0]+'.dsproj_data/FlowFM',Modellensets[0])

#### Verschil inundatieoppervlak, waterdiepte en duur van de inundatie naar shapefile

In [ ]:
DiffDiepDuur(output_pad,model_pad+'/'+Modellensets[1]+'.dsproj_data/FlowFM',Modellensets[1],model_pad+'/'+Modellensets[0]+'.dsproj_data/FlowFM',Modellensets[0])